# ${Aflow}$ Implementation experiments

## Source
* https://arxiv.org/pdf/2410.10762

## Notes
* Use gpt-4o-mini as executor/workflow constructor (executor/optimizer)
* backprop through modifications (?) that are descriptions for the executor to implement onto the parent
     * therefore need a way to modify the parent via text

## Algorithm
* Split data into 20% val/80% training
* $async$ Get scores for initial workflow on val data
* Pare down val data to examples with high score variance (above threshold)
* set $experiences = []$
* set $allresults = []$
* set $topkW = []$
* set $W^* = None$
* set $topkScores = []$
* set $bestScore = 0$

* for each training step,
    * set parent as initial workflow for first step or select parent from last training step results using subroutine ${SelectParent}$
    * get the optimizer context (parent workflow + last step experiences)
    * use the optimizer with the context and set of available operators to improve the parent workflow and generate $W_{round}$ and the ${modification}$ it's based on
    * set $evalresults=[]$
    * $async$ for each of 5 rounds
        * execute $W_{round}$ on the val data and get ${score}/{cost}$ from evaluator ${E} / {G}$
        * append the ${score},{cost}$ to $evalresults$
    * append $evalresults$ to $allresults$
    * calculate the average $score$ for the 5 rounds from $evalresults$
    * create a new experience with the parent workflow, the modification to produce $W_{round}$, and the average $score$
    * append the experience to $experiences$ 
    * if the average $score$ is higher than the $bestScore$, update $W^* = W_{round}$ and $bestScore = $ average $score$
    * add $W_{round}$ to $topkW$ and average $score$ to $topkScores$ if $topkW$ has fewer than $k$ elements or average $score$ is greater than the lowest value in $topkScores$
    * if $topkScores$ hasn't changed in specified (n) number of rounds, stop early and return $W^*$
* return $W^*$
    
### Algorithm Unknowns
* how to get evaluator G/E?
* how do initial scores get computed? are there multiple rounds of scoring per example in order to generate variance samples?
* how does the optimizer generate new workflows from previous samples (experiences)?


In [81]:
from math import e
from random import choices,seed,random
from uuid import uuid5 as uuid
from uuid import NAMESPACE_DNS
from copy import deepcopy
from pydantic import BaseModel,conint

In [71]:

# DONE: Implement create_experience
class Experience:
    def __init__(self,workflow,round_added,score,parent_score=None,modification=None):
        self.workflow = workflow
        
        self.parent_workflow = self.workflow.parent
        if self.parent_workflow:
            self.parent_workflow_id = self.parent_workflow.id
        else:
            self.parent_workflow_id = None
        self.parent_score = parent_score
        
        if modification:
            self.round_modification_added = round_added
            self.round_added = self.parent_workflow.round_added
        else:
            self.round_modification_added = None
            self.round_added = round_added
        
        self.score = score
        self.modification = modification
        
        if self.parent_score:
            self.success = self.score > self.parent_score
        else:
            self.success = None
    
    def __str__(self):
        return f"""
        Experience Info:
        Metadata:
        - Workflow with modification ID: {self.workflow.id}
        - Workflow with modification score: {self.score}
        - Workflow without modification ID: {self.parent_workflow_id}
        - Workflow without modification score: {self.parent_score}
        
        
        Modifications:
        - Lead to improvement over workflow without modifications: {self.success}
        - Round Workflow was modified: {self.round_added}
        - Modification details: {self.modification}
        """

# DONE: Implement Experience(Experiences?) obj in a way that makes retrieval by round easier
class Experiences:
    def __init__(self):
        self.experiences = {}
    
    def __getitem__(self, key):
        return self.experiences[key]
        
    def create_experience(self,workflow,round_,score,modification=None):
        if round_ not in self.experiences:
            self.experiences[round_] = {}
        parent_score = None
        if modification:
            parent_score = self.experiences[workflow.parent.round_added][workflow.parent.id].score
        self.experiences[round_][workflow.id] = Experience(workflow,round_,parent_score,modification)
        return self.experiences[round_][workflow.id]

In [67]:
class WorkflowNode:
    def __init__(self,next_node=None):
        if next_node is None:
            self.next = FinalNode()
        else:
            self.next = next_node()
        
    def __call__(self,prev_step):
        #does something with prev_step and returns it
        ...
        
    def copy(self):
        return deepcopy(self)
    
class FinalNode:
    def __init__(self):
        self.next = None
        
    def __call__(self,prev_step):
        return prev_step
    
    def copy(self):
        return self
    
class Workflow:
    #contains all of the information needed to understand a single workflow
    def __init__(self,round_added = 0,parent = None):
        self.next = FinalNode()
        self.round_added = round_added
        self.copies = 0
        # DONE: make sure workflows are being compared based on their IDs (since there may not be consistent parity between naive objects)
        self.id = uuid(NAMESPACE_DNS,str(self.round_added)+chr(97+self.copies))
        self.parent = parent
        
    def start(self,first_inputs):
        self.next(**first_inputs)
        
    def add_node(self,node_class:WorkflowNode):
        current_node = self.next
        while not isinstance(current_node,FinalNode):
            current_node = current_node.next
            
        new_node = node_class()
        current_node.next = new_node
        
    def copy(self):
        # Create a shallow copy of the current instance
        new_workflow = Workflow(self.round_added,self)
        new_workflow.next = self.next.copy()
        new_workflow.copies = self.copies + 1
        # DONE: make sure workflows are being compared based on their IDs (since there may not be consistent parity between naive objects)
        new_workflow.id = uuid(NAMESPACE_DNS, str(new_workflow.round_added) + chr(97 + new_workflow.copies))
        return new_workflow 
    
    def extract_chain_info(self):
        chain = []
        chain_info = []
        current_node = self.next
        while current_node.next:
            chain.append(type(current_node))
            chain_info.append(current_node.description)
            
        return ' -->> '.join(map(str,chain)),'\n'.join([':'.join((node,node_info)) for node,node_info in zip(map(str,chain),chain_info)])

In [70]:
# DONE: Implement split_data
def split_data(dataset,val_ratio=0.2,random_seed=1337)
    seed(random_seed)
    val_set = []
    test_set = []
    for datum in dataset:
        if random() < val_ratio:
            val_set.append(datum)
        else:
            test_set.append(datum)
            
    return val_set,test_set

In [26]:
# DONE: Implement determine_variance_threshold
# DONE: Implement select_high_variance_instances
# TODO: Handle asynchronous running (e.g. each output score set val data i may not correspond with input val data i)
def select_high_variance_instances(val_data,score_sets,marginal_variance_tolerance_perc = 0.05):
    variances = []
    n_data_points = len(score_sets[0])
    for i in n_data_points:
        scores_i = [score_set[i] for score_set in score_sets]
        num_scores_i = len(scores_i)
        avg_score_i = sum(scores_i)/num_scores_i
        sample_variance_i = (1/num_scores_i)*sum((score_i-avg_score_i)**2 for score_i in scores_i)
        variances.append(sample_variance_i)
        
    sorted_variances = sorted(list(set(variances)),reversed=True)
    #use elbow method diminishing gains based on marginal_variance_tolerance_perc
    last_variance = None
    for variance in sorted_variances:
        if last_variance is None:
            last_variance = variance
        else:
            perc_change = (variance/last_variance)-1
            if perc_change < marginal_variance_tolerance_perc:
                return last_variance
            last_variance = variance
            
    return [datum for i,datum in enumerate(val_data) if variances[i]>last_variance],last_variance
            

In [28]:
# DONE: Implement select_parent
def select_parent(topk_scores,topk_W):
    probabilities = calculate_mixed_probabilities(topk_scores) #DONE: Implement calculate_mixed_probabilities
    return sample_from_categorical(probabilities,topk_W) #DONE: Implement sample_from_categorical

In [29]:
#DONE: Implement calculate_mixed_probabilities
def calculate_mixed_probabilities(scores,lambda_=0.4,alpha=0.5): #TODO: Determine appropriate values of lambda and alpha from the paper
    n = len(scores)
    max_score = max(scores)
    w = (e**(alpha*(s_i-max_score)) for s_i in scores)
    P_score = (w_i/sum(w) for w_i in w)
    P_uniform = (1/n for _ in range(n))
    P_mixed = (lambda_ * P_uniform_i + (1-lambda_) * P_score_i for P_score_i,P_uniform_i in zip(P_score,P_uniform))
    return P_mixed

In [30]:
#DONE: Implement sample_from_categorical
def sample_from_categorical(probabilities,workflows):
    return random.choices(population=workflows,weights=probabilities,k=1)

In [75]:
# DONE: Implement load_context
def load_context(parent_workflow,experiences):
    #get the experiences related to parent_workflow
    #parent_workflow may or may not have modifications
    experience_info = []
    for round_ in experiences:
        if parent_workflow.id in experiences[round_]:
            experience_info.append(str(experiences[round_][parent_workflow.id]))
    workflow_experiences = '\n\n'.join(experience_info)
    
    #get the node chain visualization and node descriptions parent_workflow implements
    chain_viz,chain_descriptions = parent_workflow.extract_chain_info()
    
    context_string = f"""
        Context for workflow {parent_workflow.id}:
        
        Workflow node chain visualization:
        {chain_viz}
        
        Descriptions of each node:
        {chain_descriptions}
        
        Trials (experiences) involving this workflow:
        {workflow_experiences}
    """
    return context_string
    
            


In [78]:
# DONE: Implement optimize
class NodeAdd(BaseModel):
    node:str
    modification:str
    
def optimize(parent_workflow,context_string,dict_of_valid_operations,llm_client):
    C,O = context_string,'\n\n'.join(map(str,dict_of_valid_operations.keys())) #TODO: Implement Node __str__ for this purpose
    
    optimize_system_prompt = """
    You are being given a chance to improve the workflow used to answer user queries.
    You will be given information about the workflows as well as information about, and results of, previous attempts (if applicable) to improve the workflow.
    You will also be given information about valid operations that can be added to the workflow in order to increase its ability to successfully answer the user query.
    
    You will be expected to reason step-by-step in order to come to a conclusion about what node (valid operation) to add to the workflow.
    
    At the end of your reasoning steps, you should output a single json object in the following format:
    {"node":"<NAME OF NODE>","modification":"Add a new <NAME OF NODE> to <AND SO ON>"}
    where "<NAME OF NODE>" is one of given node names. If you fail to choose an existing node name, you will fail your task.
    And where <AND SO ON> is where the rest of your explanation should go. If you fail to rationally, simply and clearly describe 
    your reasoning here, you will fail your task and future optimizations will go poorly.
    
    You are OPTIMIZER.
    """
    
    optimize_user_prompt = f"""
    Here is your context:
    Workflow information:
    {C}
    
    Valid operations/nodes:
    {O}
    """
    
    client_response = llm_client.beta.chat.completions.parse(
        messages = [
            {"role":"system","content":optimize_system_prompt},
            {"role":"user"  ,"content":optimize_user_prompt}
        ],
        model='gpt-4o-mini'
        response_format=NodeAdd,
        temperature = 0.0
    )
    
    nodeadd_obj = client_response.choices[0].message.parsed
    W_round = parent_workflow.copy().add_node(dict_of_valid_operations(nodeadd_obj['node']))
    return W_round,nodeadd_obj['modification']

In [ ]:
#DONE: Implement evaluator 
#will use gpt-4o to assess results (small dataset for now)
class Evaluation(BaseModel):
    grade:conint(ge=0,le=10)
        
def evaluate(question,correct_answer,given_answer):
    evaluate_system_prompt = """
    You are being given a chance to evaluate the result of a workflow used to answer user queries.
    You will be given the user query, the correct answer, the the answer provided by the workflow.
    
    You will be expected to succinctly explain any issues with the answer from a content and understandability perspective.
    
    At the end of your succinct explanation, you should output a single json object in the following format:
    {"grade":<GRADE>}
    where <GRADE> is an integer between 0 and 10, where 0 is completely irrelevant and/or completely unintelligible, and 10 is perfect in both content and understandability. In between is a continuum of integer grades describing differing levels of imperfection. 
    
    You are EVALUATOR.
    """
    
    evaluate_user_prompt = f"""
    Here is the user query:
    {question}
    
    Here is the correct answer:
    {correct_answer}
    
    Here is the answer generated by the workflow:
    {given_answer}
    """
    
    client_response = llm_client.beta.chat.completions.parse(
        messages = [
            {"role":"system","content":evaluate_system_prompt},
            {"role":"user"  ,"content":evaluate_user_prompt}
        ],
        model='gpt-4o'
        response_format=Evaluation,
        temperature = 0.0
    )
    
    evaluation_obj = client_response.choices[0].message.parsed
    return evaluation_obj.grade

In [79]:
# DONE: Implement execute_workflow_over_data
# TODO: Make this asynchronous
# TODO: Implement categorical loss
def execute_workflow_over_data(workflow,evaluator,data,llm_client):
    scores = []
    costs = []
    for datum in data:
        q,a = datum
        a_hat,cost = workflow.start({'data':q,'llm_client':llm_client})
        score = evaluator(q,a,a_hat) #this works for a continuous loss/gain (e.g. answer quality/lack of), TODO: Implement categorical loss
        scores.append(score)
        costs.append(cost)
    return scores,sum(costs)
        

In [80]:
def get_optimal_workflow(initial_workflow,evaluator,dataset,operators,llm_client,number_of_rounds=20,k=3,early_stopping_rounds=5,scoring_rounds=5):
    #DONE: Implement evaluator
    #Initialize variables
    W_0, G, D, N, O, k, n, I = initial_workflow,evaluator,dataset,number_of_rounds,operators,k, \
                                early_stopping_rounds,scoring_rounds
    results = []
    experiences = Experiences()
    topk_W = []
    best_W = None
    topk_scores = []
    best_score = 0
    topk_W_unchanged = 0
    
    #Split dataset
    Dv,Dt = split_data(dataset) #DONE: Implement split_data
    
    #Gather initial scores to determine final validation dataset
    score_sets,_ = zip(*[execute_workflow_over_data(W_0,G,Dv) for _ in range(k)]) #DONE: Implement execute_workflow_over_data
    Dv,variance_threshold = select_high_variance_instances(Dv,score_sets)   #DONE: Implement select_high_variance_instances
                                                                        #DONE: Implement determine_variance_threshold
    
    print(f'{len(Dv)} validation samples selected based on variance threshold {variance_threshold:.2f}')
    #MAIN LOOP----------------------------------------
    #Iterate to improve bestScore
    for round_ in range(len(N)):
        if round_ == 1:
            parent = W_0
        else:
            parent = select_parent(topk_scores,topk_W) #DONE: Implement select_parent
        
        #Load context for parent and perform optimization forward pass
        context = load_context(parent,experiences) #DONE: Implement load_context
        W_round, modification = optimize(parent,context,O,llm_client) #DONE: Implement optimize
        
        #Generate validation scores for modified workflow to demonstrate relative performance
        for i in range(I):
            scores,cost = execute_workflow_over_data(W_round,G,Dv,llm_client) #DONE: Implement execute_workflow_over_data
            round_score = sum(scores)/len(scores) #this works for a continuous loss/gain (e.g. answer quality/lack of), TODO: Implement categorical loss
            results.append((round_,round_score,cost))
        
        #Capture a new experience for use in future optimization passes by using the modified workflow validation scores as gain
        average_score = sum(r[1] for r in results if results[0]==round_)/len(r[1] for r in results if results[0]==round_)
        experience = experiences.create_experience(W_round,round_,average_score,modification) #DONE: Implement create_experience
#         experiences.append(experience) #DONE: Implement Experience(Experiences?) obj in a way that makes retrieval by round easier
        
        #Save the previous topk_W for the later early-stopping check
        prev_topk_W = topk_W
        
        #Update best W, if applicable this round
        if average_score > best_score:
            best_score = average_score
            best_W = W_round
            
            topk_W.append(W_round)
            topk_scores.append(average_score)
        
        #Try to include this round's W_round into topk_W if the W_round's score is within the top k scores
        combined = sorted(zip(topk_scores,topk_W),reverse=True)
        topk_scores[:],topk_W[:] = zip(*combined)
        
        #Boot any workflows outside of the top k scores
        if len(topk_scores)>k:
            topk_scores = list(topk_scores[:-1])
            topk_W = list(topk_W[:-1])
            
        #Check if topk_W is the same as the prev topk_W and increment the early stopping counter if so. If not, set counter to 0.
        if set([W.id for W in topk_W])==set([prevW.id for prevW in prev_topk_W]):
            topk_W_unchanged+=1 #DONE: make sure workflows are being compared based on their IDs 
                                                    #(since there may not be consistent parity between naive objects)
        else:
            topk_W_unchanged = 0
        
        #Stop the workflow improvement early if no new workflows have joined the top k workflows in n rounds
        if topk_W_unchanged >= n:
            break
    #END MAIN LOOP------------------------------------
    #return the best workflow and the k-1 next best workflows/the best workflow's score and the k-1 next best workflows' scores
    return (best_W,topk_W),(best_score,topk_scores)
    

In [48]:
initial_workflow = Workflow()
operators = [] #registered nodes to be used in optimization

In [49]:
initial_workflow.__dict__

{'next': <__main__.FinalNode at 0x264ff72f888>,
 'round_added': 0,
 'copies': 0,
 'id': UUID('81060c4f-efaf-54d6-b722-5509dce87911')}

In [50]:
initial_workflow.copy().__dict__

{'next': <__main__.FinalNode at 0x264ff72f888>,
 'round_added': 0,
 'copies': 1,
 'id': UUID('4cf211f1-3335-5c59-a280-9ad90e75b23e')}

### Future Enhancements
We can optimize one layer/level of operators.

What about two layers of operators? (e.g. operators nested in operators)

What about N layers of operators?


e.g. can we implement hierarchical optimization so that each and every step in the chain is tightly fit to purpose

e.g. can we construct operators from smaller units on the fly?

e.g. are there general operators such that these operators can be constructed of these operators at any level of depth


We have implemented breadth (operator columns) - can we implement depth (operator rows)?

Can we do it just as fast as breadth?